### Initiate Environment

In [1]:
using Pkg
Pkg.activate(".")

using Revise
using TitanicClassifier
using CSV
using DataFrames

  Activating project at `~/CTU/SEM_5/JUL/TitanicClassifier/examples`


### Data Analysis

In [2]:
data = CSV.read("../data/train.csv", DataFrame);

In [64]:
incomplete_cols = Dict()

for row in eachrow(data)
    for name in names(data)
        if(ismissing(row[name]))
            incomplete_cols[name] = get!(incomplete_cols, name, 0) + 1
        end
    end
end

for entry in incomplete_cols
    print("Missing ", entry[2], " out of ", nrow(data), " entries (", round(entry[2]/nrow(data); sigdigits=2), "%) for column ", entry[1], "\n")
end

Missing 177 out of 891 entries (0.2%) for column Age
Missing 2 out of 891 entries (0.0022%) for column Embarked
Missing 687 out of 891 entries (0.77%) for column Cabin


We need to fill missing columns with some data in order for our model to work. For cabin the number of missing entries
is great and there is no clear method to impute the missing data - it might be possible to infer the cabin data from ticket 
and embarked features, however I chose just to introduce an unknown token for the missing entries. Additionally since one of the sources for cabin data is the recollection of survivors it seems that unknown token holds information for survival inference. Age and embarked feature I will impute from the other features.

### Data Preprocessing

In [9]:
freq = title_frequencies(data)

Dict{Any, Any} with 17 entries:
  "Jonkheer"     => 1
  "Don"          => 1
  "the Countess" => 1
  "Master"       => 40
  "Miss"         => 182
  "Capt"         => 1
  "Mrs"          => 125
  "Rev"          => 6
  "Major"        => 2
  "Mr"           => 517
  "Mme"          => 1
  "Mlle"         => 2
  "Col"          => 2
  "Lady"         => 1
  "Sir"          => 1
  "Ms"           => 1
  "Dr"           => 7

In [4]:
replace_rules = [[["Dr", "Rev", "Col", "Major", "Capt"], "Officer"],
                 [["Jonkheer", "Countess", "Sir", "Lady", "Don"], "Royalty"], 
                 [["Mlle"], "Miss"], [["Ms"], "Miss"], [["Mme"],"Mrs"]]

Mr

In [5]:
function missing_filter(df)
    for col in df
        if ismissing(col)
            return false
        end
    end
    return true
end

filter(missing_filter, data[1:5,:])

Row,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
,Int64,Int64,Int64,String,String7,Float64?,Int64,Int64,String31,Float64,String15?,String1?
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Thayer)",female,38.0,1,0,PC 17599,71.2833,C85,C
2,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1,C123,S


In [6]:
extract_deck_from_cabin(cabin) = cabin[1]

decks = Dict()

id = 1
for row in eachrow(data)
    if !ismissing(row["Cabin"]) && !haskey(decks, extract_deck_from_cabin(row["Cabin"]))
        decks[extract_deck_from_cabin(row["Cabin"])] = id
        id+=1
    end
end
decks['U'] = id
print(decks)

Dict{Any, Any}('B' => 6, 'U' => 9, 'D' => 4, 'A' => 5, 'E' => 2, 'G' => 3, 'T' => 8, 'F' => 7, 'C' => 1)

In [7]:
embarks = Dict()

id = 1
for row in eachrow(data)
    if !ismissing(row["Embarked"]) && !haskey(embarks, row["Embarked"])
        embarks[row["Embarked"]] = id
        id+=1
    end
end
print(embarks)

Dict{Any, Any}(

String1("Q") => 3, String1("S") => 1, String1("C") => 2)

In [8]:
tickets = Set()

function extract_ticket_num(ticket) 
    temp = split(ticket, " ")
    if (length(temp) == 1 && temp[1]=="LINE")
        return -1
    else 
        return parse(Int64, last(temp))
    end
end

for row in eachrow(data)
    push!(tickets, extract_ticket_num(row["Ticket"]))
end

sorted_nums = sort(collect(tickets))

ticket_idx = Dict()
for i in 1:length(sorted_nums)
    ticket_idx[sorted_nums[i]] = i
end

In [18]:

title_tokens = Dict()

i = 0

replace_rules = [[["Dr", "Rev", "Col", "Major", "Capt"], "Officer"],
[["Jonkheer", "Countess", "Sir", "Lady", "Don"], "Royalty"], 
[["Mlle"], "Miss"], [["Ms"], "Miss"], [["Mme"],"Mrs"]]

for (title, value) in freq
    if !haskey(title_tokens, get_title_token(title, replace_rules))
        if(get_title_token(title, replace_rules)=="Unknown")
            print(title)
        end
        title_tokens[get_title_token(title, replace_rules)] = i
        i+=1
    end
end

print(title_tokens)

Dict{Any, Any}(

"Miss" => 2, "Master" => 1, "Officer" => 3, "Royalty" => 0, "Mrs" => 4, "Mr" => 5)

In [19]:
processed_data = copy(data)

enumerate_sex(str) = str == "male" ? 0 : 1

#print(processed_data[!, 5])
#mapcols!(enumerate_sex, processed_data[!, 5])
transform!(processed_data, :Sex => enumerate_sex => :Sex)
transform!(processed_data, :Cabin => ByRow(cabin -> ismissing(cabin) ? decks['U'] : decks[extract_deck_from_cabin(cabin)]) => :Cabin)
transform!(processed_data, :Ticket => ByRow(ticket -> ticket_idx[extract_ticket_num(ticket)]) => :Ticket)
transform!(processed_data, :Embarked => ByRow(embarked -> ismissing(embarked) ? missing : embarks[embarked]) => :Embarked)
transform!(processed_data, :Name => ByRow(name -> title_tokens[get_title_token(name, replace_rules)]) => :Name)


print(first(processed_data, 5))

InterruptException: InterruptException: